In [1]:
import pandas as pd
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import itertools
import random

# adjusting display options
pd.set_option('display.max_colwidth', None)

data = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')
n_rows = data.shape[0]
data

,id,text
0,0,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
1,1,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
2,2,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
3,3,yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
4,4,hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle
5,5,advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle


In [2]:
# Step 3: Define Function to Calculate Perplexity
def calculate_perplexity(text, model, tokenizer):
    encodings = tokenizer(text, return_tensors="pt")
    max_length = model.config.n_positions
    stride = max_length

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = i
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - begin_loc
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
        nlls.append(neg_log_likelihood)
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl.item()

In [3]:
# Step 4: Load GPT-2 Model and Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Define a function to calculate the perplexity of a given sequence
def calculate_perplexity(text, model, tokenizer):
    encodings = tokenizer(text, return_tensors="pt")
    max_length = model.config.n_positions
    stride = max_length

    nlls = []
    for i in range(0, encodings.input_ids.size(1), stride):
        begin_loc = i
        end_loc = min(i + stride, encodings.input_ids.size(1))
        trg_len = end_loc - begin_loc
        input_ids = encodings.input_ids[:, begin_loc:end_loc]
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs.loss
        nlls.append(neg_log_likelihood)
    ppl = torch.exp(torch.stack(nlls).mean())
    return ppl.item()

In [5]:
# Function to optimize a single sequence
def optimize_sequence(base_text, model, tokenizer, num_permutations=100):
    words = base_text.split()  # Split the base text into words
    best_perplexity = float("inf")  # Start with a very high perplexity
    best_sequence = base_text  # Initialize with the base text

    # Generate random permutations
    for _ in range(num_permutations):
        random.shuffle(words)  # Shuffle the words randomly
        permuted_text = " ".join(words)  # Join the shuffled words into a sentence
        current_perplexity = calculate_perplexity(permuted_text, model, tokenizer)
        if current_perplexity < best_perplexity:
            best_perplexity = current_perplexity
            best_sequence = permuted_text

    return best_sequence, best_perplexity  # Return both sequence and perplexity

In [6]:
# Initialize lists to store results
optimized_texts = []
perplexities = []

# Iterate through each row in the dataset
for index, row in data.iterrows():
    base_text = row['text']  # Extract the base text
    print(f"Optimizing sequence for ID {row['id']}...")
    optimized_text, perplexity = optimize_sequence(base_text, model, tokenizer)
    optimized_texts.append(optimized_text)  # Save the optimized sequence
    perplexities.append(perplexity)  # Save the corresponding perplexity

    print(f"Completed ID {row['id']}: Perplexity = {perplexity}")

Optimizing sequence for ID 0...
Completed ID 0: Perplexity = 329.2959899902344
Optimizing sequence for ID 1...
Completed ID 1: Perplexity = 771.3868408203125
Optimizing sequence for ID 2...
Completed ID 2: Perplexity = 754.3718872070312
Optimizing sequence for ID 3...
Completed ID 3: Perplexity = 986.8521728515625
Optimizing sequence for ID 4...
Completed ID 4: Perplexity = 910.3346557617188
Optimizing sequence for ID 5...
Completed ID 5: Perplexity = 726.77978515625


In [7]:
# Add the optimized texts to the DataFrame
data['text'] = optimized_texts

# Print a summary
print(f"Optimization completed for {len(data)} rows.")

Optimization completed for 6 rows.


In [8]:
# Save the submission file
submission_file = "santa_submission_v1.2.csv"
data[['id', 'text']].to_csv(submission_file, index=False)  # Exclude perplexity column
print(f"Submission file saved as {submission_file}")

Submission file saved as santa_submission_v1.2.csv
